### Import Libraries

In [1]:
from modelTrainer import ModelTrainer
from metrics import recall, precision, f1Score

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Common Variables

In [2]:
# path to data on aws
data_path = "/home/ubuntu/project/data/groundcover2016/"
model_path = "/home/ubuntu/project/model_checkpoints/unet/"
results_path = "/home/ubuntu/project/results/"
modelTrainer = ModelTrainer(data_path,results_path,model_path)
modelTrainer.changeMetrics(['acc',recall,precision,f1Score])

classMap = {
    'maize' : 1,
    'maizevariety': 1,
    'wheat': 2,
    'mungbean': 3
}

## Single Class Model (Initial Depth = 64)

In [ ]:
modelTrainer.changeEpochs(2)
modelTrainer.changeBatchSize(18)
modelTrainer.changeConvolutionalDepth(64)
modelTrainer.setWeightInitializer('he_normal')
modelTrainer.setOptimizerParams(lr = 1.0*(10**-4),momentum = 0.8,decay = 1.0*(10**-8))
modelTrainer.changeDropout(0.6)
modelTrainer.setClassName("maize")
modelTrainer.setSaveName("maize64")
modelTrainer.train()

----------Model Parameters----------
Initial Conv. Depth : 64
Number of Classes   : 2
Dropout             : 0.6
Activation Function : Relu
Input Shape         : (256, 256, 3)
Batch Size          : 18
--------Optimizer Parameters--------
Learning Rate : <tf.Variable 'SGD/lr:0' shape=() dtype=float32_ref>
Momentum      : <tf.Variable 'SGD/momentum:0' shape=() dtype=float32_ref>
Initial Decay : 1e-08
Epoch 1/2
   5/3955 [..............................] - ETA: 3:53:56 - loss: 1.0040 - acc: 0.2279 - recall: 0.2279 - precision: 0.2279 - f1Score: 0.2279

## Single Class Model (Initial Depth = 32)

In [4]:
modelTrainer.changeEpochs(2)
modelTrainer.changeBatchSize(32)
modelTrainer.changeConvolutionalDepth(32)
modelTrainer.setWeightInitializer('he_normal')
modelTrainer.setOptimizerParams(lr = 1.0*(10**-3),momentum = 0.8,decay = 1.0*(10**-8))
modelTrainer.changeDropout(0.6)
modelTrainer.setClassName("maize")
modelTrainer.setSaveName("maize32")
modelTrainer.train()

----------Model Parameters----------
Initial Conv. Depth : 32
Number of Classes   : 2
Dropout             : 0.6
Activation Function : Relu
Input Shape         : (256, 256, 3)
Batch Size          : 32
--------Optimizer Parameters--------
Learning Rate : <tf.Variable 'SGD/lr:0' shape=() dtype=float32_ref>
Momentum      : <tf.Variable 'SGD/momentum:0' shape=() dtype=float32_ref>
Initial Decay : 1e-08
Epoch 1/2
2225/2225 [==============================] - 3167s 1s/step - loss: 0.1524 - acc: 0.9337 - recall: 0.9337 - precision: 0.9337 - f1Score: 0.9337 - val_loss: 0.1360 - val_acc: 0.9399 - val_recall: 0.9399 - val_precision: 0.9399 - val_f1Score: 0.9399

Epoch 00001: saving model to /home/ubuntu/project/model_checkpoints/unet//maize32.hdf5
Epoch 2/2
2225/2225 [==============================] - 3160s 1s/step - loss: 0.1395 - acc: 0.9384 - recall: 0.9384 - precision: 0.9384 - f1Score: 0.9384 - val_loss: 0.1318 - val_acc: 0.9413 - val_recall: 0.9413 - val_precision: 0.9413 - val_f1Score: 0.94

## Single Class Model (Initial Depth = 16)

In [6]:
modelTrainer.changeBatchLogInterval(5)
modelTrainer.changeEpochs(2)
modelTrainer.changeDropout(0.6)
modelTrainer.setWeightInitializer('he_normal')
modelTrainer.changeConvolutionalDepth(16)
modelTrainer.changeBatchSize(64)
modelTrainer.setClassName("maize")
modelTrainer.setOptimizerParams(lr = 1.0*(10**-3),momentum = 0.9,decay = 1.0*(10**-6))
modelTrainer.setSaveName("maize16")
modelTrainer.train()

----------Model Parameters----------
Initial Conv. Depth : 16
Number of Classes   : 2
Dropout             : 0.6
Activation Function : Relu
Input Shape         : (256, 256, 3)
Batch Size          : 64
--------Optimizer Parameters--------
Learning Rate : <tf.Variable 'SGD_2/lr:0' shape=() dtype=float32_ref>
Momentum      : <tf.Variable 'SGD_2/momentum:0' shape=() dtype=float32_ref>
Initial Decay : 1e-06
Epoch 1/2
1112/1112 [==============================] - 1864s 2s/step - loss: 0.1601 - acc: 0.9299 - recall: 0.9299 - precision: 0.9299 - f1Score: 0.9299 - val_loss: 0.1329 - val_acc: 0.9413 - val_recall: 0.9413 - val_precision: 0.9413 - val_f1Score: 0.9413

Epoch 00001: saving model to /home/ubuntu/project/model_checkpoints/unet//maize16.hdf5
Epoch 2/2
1112/1112 [==============================] - 1859s 2s/step - loss: 0.1386 - acc: 0.9388 - recall: 0.9388 - precision: 0.9388 - f1Score: 0.9388 - val_loss: 0.1318 - val_acc: 0.9412 - val_recall: 0.9412 - val_precision: 0.9412 - val_f1Score: 

## Single Class Model (Initial Depth = 8)

In [3]:
modelTrainer.changeBatchLogInterval(5)
modelTrainer.changeEpochs(2)
modelTrainer.changeDropout(0.6)
modelTrainer.setWeightInitializer('he_normal')
modelTrainer.changeConvolutionalDepth(8)
modelTrainer.changeBatchSize(128)
modelTrainer.setClassName("maize")
modelTrainer.setOptimizerParams(lr = 1.0*(10**-3),momentum = 0.9,decay = 1.0*(10**-6))
modelTrainer.setSaveName("maize8")
modelTrainer.train()

----------Model Parameters----------
Initial Conv. Depth : 8
Number of Classes   : 2
Dropout             : 0.6
Activation Function : Relu
Input Shape         : (256, 256, 3)
Batch Size          : 128
--------Optimizer Parameters--------
Learning Rate : <tf.Variable 'SGD/lr:0' shape=() dtype=float32_ref>
Momentum      : <tf.Variable 'SGD/momentum:0' shape=() dtype=float32_ref>
Initial Decay : 1e-06
Epoch 1/2
556/556 [==============================] - 1360s 2s/step - loss: 0.1931 - acc: 0.9183 - recall: 0.9183 - precision: 0.9185 - f1Score: 0.9184 - val_loss: 0.1730 - val_acc: 0.9224 - val_recall: 0.9224 - val_precision: 0.9224 - val_f1Score: 0.9224

Epoch 00001: saving model to /home/ubuntu/project/model_checkpoints/unet//maize8.hdf5
Epoch 2/2
556/556 [==============================] - 1331s 2s/step - loss: 0.1444 - acc: 0.9374 - recall: 0.9374 - precision: 0.9374 - f1Score: 0.9374 - val_loss: 0.1338 - val_acc: 0.9414 - val_recall: 0.9414 - val_precision: 0.9414 - val_f1Score: 0.9414

E

## Single Class Model (Initial Depth = 4)

In [7]:
modelTrainer.changeBatchLogInterval(5)
modelTrainer.changeEpochs(3)
modelTrainer.changeDropout(0.6)
modelTrainer.setWeightInitializer('he_normal')
modelTrainer.changeConvolutionalDepth(4)
modelTrainer.changeBatchSize(256)
modelTrainer.setClassName("maize")
modelTrainer.setOptimizerParams(lr = 1.0*(10**-3),momentum = 0.9,decay = 1.0*(10**-6))
modelTrainer.setSaveName("maize4")
modelTrainer.train()

----------Model Parameters----------
Initial Conv. Depth : 4
Number of Classes   : 2
Dropout             : 0.6
Activation Function : Relu
Input Shape         : [256, 256, 3]
Batch Size          : 256
--------Optimizer Parameters--------
Learning Rate : <tf.Variable 'SGD_5/lr:0' shape=() dtype=float32_ref>
Momentum      : <tf.Variable 'SGD_5/momentum:0' shape=() dtype=float32_ref>
Initial Decay : 1e-06
Epoch 1/3
278/278 [==============================] - 1133s 4s/step - loss: 0.4875 - acc: 0.8185 - recall: 0.8185 - precision: 0.8185 - f1Score: 0.8185 - val_loss: 0.4130 - val_acc: 0.8445 - val_recall: 0.8445 - val_precision: 0.8445 - val_f1Score: 0.8445

Epoch 00001: saving model to /home/ubuntu/project/model_checkpoints/unet//maize4.hdf5
Epoch 2/3
278/278 [==============================] - 1128s 4s/step - loss: 0.4154 - acc: 0.8210 - recall: 0.8210 - precision: 0.8210 - f1Score: 0.8210 - val_loss: 0.3328 - val_acc: 0.8448 - val_recall: 0.8448 - val_precision: 0.8448 - val_f1Score: 0.844

# Train a multi-class model (Initial Depth = 64)

In [ ]:
modelTrainer.changeBatchLogInterval(5)
modelTrainer.changeEpochs(1)
modelTrainer.changeDropout(0.6)
modelTrainer.setWeightInitializer('he_normal')
modelTrainer.changeConvolutionalDepth(64)
modelTrainer.changeBatchSize(16)
modelTrainer.setClassMap(classMap)
modelTrainer.setOptimizerParams(lr = 1.0*(10**-4),momentum = 0.9,decay = 1.0*(10**-10))
modelTrainer.setSaveName("multiclass64")
modelTrainer.train()

----------Model Parameters----------
Initial Conv. Depth : 64
Number of Classes   : 4
Dropout             : 0.6
Activation Function : Relu
Input Shape         : (256, 256, 3)
Batch Size          : 16
--------Optimizer Parameters--------
Learning Rate : <tf.Variable 'SGD/lr:0' shape=() dtype=float32_ref>
Momentum      : <tf.Variable 'SGD/momentum:0' shape=() dtype=float32_ref>
Initial Decay : 1e-10
Epoch 1/1
 8290/20509 [===========>..................] - ETA: 5:22:39 - loss: 0.4423 - acc: 0.8263 - recall: 0.7530 - precision: 0.9055 - f1Score: 0.8206

## Multi-Class model (Initial Depth = 32) 

In [4]:
modelTrainer.changeBatchLogInterval(5)
modelTrainer.changeEpochs(1)
modelTrainer.changeDropout(0.6)
modelTrainer.setWeightInitializer('he_normal')
modelTrainer.changeConvolutionalDepth(32)
modelTrainer.changeBatchSize(36)
modelTrainer.setClassMap(classMap)
modelTrainer.setOptimizerParams(lr = 1.0*(10**-4),momentum = 0.9,decay = 1.0*(10**-10))
modelTrainer.setSaveName("multiclass32")
modelTrainer.train()

----------Model Parameters----------
Initial Conv. Depth : 32
Number of Classes   : 4
Dropout             : 0.6
Activation Function : Relu
Input Shape         : (256, 256, 3)
Batch Size          : 36
--------Optimizer Parameters--------
Learning Rate : <tf.Variable 'SGD/lr:0' shape=() dtype=float32_ref>
Momentum      : <tf.Variable 'SGD/momentum:0' shape=() dtype=float32_ref>
Initial Decay : 1e-10
Epoch 1/5
9115/9115 [==============================] - 17138s 2s/step - loss: 0.4427 - acc: 0.8257 - recall: 0.7537 - precision: 0.9072 - f1Score: 0.8223 - val_loss: 0.3977 - val_acc: 0.8428 - val_recall: 0.7899 - val_precision: 0.9001 - val_f1Score: 0.8413

Epoch 00001: saving model to /home/ubuntu/project/model_checkpoints/unet//smallMulticlassNetwork.hdf5
Epoch 2/5
 232/9115 [..............................] - ETA: 4:12:07 - loss: 0.3986 - acc: 0.8435 - recall: 0.7889 - precision: 0.8998 - f1Score: 0.8406

KeyboardInterrupt: 

### Train multi-class model for another epoch

In [3]:
modelTrainer.changeBatchLogInterval(5)
modelTrainer.changeEpochs(1)
modelTrainer.changeDropout(0.6)
modelTrainer.setWeightInitializer('he_normal')
modelTrainer.changeBatchSize(36)
modelTrainer.setClassMap(classMap)
modelTrainer.setOptimizerParams(lr = 1.0*(10**-8),momentum = 0,decay =0)
modelTrainer.setSaveName("multiclass32")

In [4]:
modelTrainer.trainMore("multiclass32")

----------Model Parameters----------
Initial Conv. Depth : 32
Number of Classes   : 4
Dropout             : 0.6
Activation Function : Relu
Input Shape         : (256, 256, 3)
Batch Size          : 36
--------Optimizer Parameters--------
Learning Rate : <tf.Variable 'SGD/lr:0' shape=() dtype=float32_ref>
Momentum      : <tf.Variable 'SGD/momentum:0' shape=() dtype=float32_ref>
Initial Decay : 0
Epoch 1/1
9115/9115 [==============================] - 16374s 2s/step - loss: 0.3918 - acc: 0.8460 - recall: 0.7930 - precision: 0.9011 - f1Score: 0.8435 - val_loss: 0.3987 - val_acc: 0.8423 - val_recall: 0.7895 - val_precision: 0.8998 - val_f1Score: 0.8409

Epoch 00001: saving model to /home/ubuntu/project/model_checkpoints/unet//smallMulticlassNetworkEpoch2.hdf5


## Multi-class model (Initial Depth = 16)

In [ ]:
classMap = {
    'maize' : 1,
    'maizevariety': 1,
    'wheat': 2,
    'mungbean': 3
}

modelTrainer.changeBatchLogInterval(5)
modelTrainer.changeEpochs(1)
modelTrainer.changeDropout(0.6)
modelTrainer.setWeightInitializer('he_normal')
modelTrainer.changeConvolutionalDepth(16)
modelTrainer.changeBatchSize(64)
modelTrainer.setClassMap(classMap)
modelTrainer.setOptimizerParams(lr = 1.0*(10**-4),momentum = 0.9,decay = 1.0*(10**-10))
modelTrainer.setSaveName("multiclass16")
modelTrainer.train()

----------Model Parameters----------
Initial Conv. Depth : 16
Number of Classes   : 4
Dropout             : 0.6
Activation Function : Relu
Input Shape         : (256, 256, 3)
Batch Size          : 64
--------Optimizer Parameters--------
Learning Rate : <tf.Variable 'SGD/lr:0' shape=() dtype=float32_ref>
Momentum      : <tf.Variable 'SGD/momentum:0' shape=() dtype=float32_ref>
Initial Decay : 1e-10
Epoch 1/1
3601/5127 [====================>.........] - ETA: 1:12:16 - loss: 0.5633 - acc: 0.7735 - recall: 0.6801 - precision: 0.9050 - f1Score: 0.7746

In [3]:
modelTrainer.setClassMap(classMap)
modelTrainer.setOldModel("unet_multiclass")
modelTrainer.changeBatchSize(48)
modelTrainer.evaluate(validationOnly = True)

----------Model Parameters----------
Initial Conv. Depth : 64
Number of Classes   : 4
Dropout             : 0
Activation Function : Relu
Input Shape         : [256, 256, 3]
Batch Size          : 48
--------Optimizer Parameters--------
Learning Rate : <tf.Variable 'SGD_1/lr:0' shape=() dtype=float32_ref>
Momentum      : <tf.Variable 'SGD_1/momentum:0' shape=() dtype=float32_ref>
Initial Decay : 0
857/857 [==============================] - 1631s 2s/step
val_loss : 0.26233535304300426
val_acc : 0.8882396911100301
val_recall : 0.8761578648760828
val_precision : 0.900224934520076
val_f1Score : 0.8880166963629572


In [3]:
#from keras.models import load_model
#m = load_model(model_path + "maize_model.hdf5")
#m.summary()
modelTrainer.setClassMap({"maize" : 1})
modelTrainer.setOldModel("maize_model")
modelTrainer.changeBatchSize(48)
modelTrainer.evaluate(validationOnly = True)

----------Model Parameters----------
Initial Conv. Depth : 64
Number of Classes   : 2
Dropout             : 0
Activation Function : Relu
Input Shape         : [256, 256, 3]
Batch Size          : 48
--------Optimizer Parameters--------
Learning Rate : <tf.Variable 'SGD_1/lr:0' shape=() dtype=float32_ref>
Momentum      : <tf.Variable 'SGD_1/momentum:0' shape=() dtype=float32_ref>
Initial Decay : 0
184/184 [==============================] - 321s 2s/step
val_loss : 7.863147219885951
val_acc : 0.0579026694935949
val_recall : 0.057902666051750595
val_precision : 0.05790266674011946
val_f1Score : 0.057902666395935026


In [3]:
modelTrainer.setClassMap({"maize" : 1})
modelTrainer.setOldModel("maize4")
modelTrainer.changeBatchSize(48)
modelTrainer.evaluate(validationOnly = True)

----------Model Parameters----------
Initial Conv. Depth : 4
Number of Classes   : 2
Dropout             : 0
Activation Function : Relu
Input Shape         : [256, 256, 3]
Batch Size          : 48
--------Optimizer Parameters--------
Learning Rate : <tf.Variable 'SGD_1/lr:0' shape=() dtype=float32_ref>
Momentum      : <tf.Variable 'SGD_1/momentum:0' shape=() dtype=float32_ref>
Initial Decay : 0
184/184 [==============================] - 123s 671ms/step
val_loss : 7.764132841773655
val_acc : 0.05984660347356745
val_recall : 0.05984660347356745
val_precision : 0.05984660371652116
val_f1Score : 0.05984660377725959
